In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.grid_search import GridSearchCV
import xgboost
%matplotlib inline

from subprocess import check_output
print(check_output(["ls", "input/"]).decode("utf8"))

/n/home12/rebecca.krall/.conda/envs/py3xg/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/n/home12/rebecca.krall/.conda/envs/py3xg/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


fastest_routes_test.csv
fastest_routes_train_part_1.csv
fastest_routes_train_part_2.csv
test.csv
train.csv
weather_data_nyc_centralpark_2016.csv



## XGBoost

In [2]:
df = pd.read_csv('train_data.csv')
tdf = pd.read_csv('test_data.csv')

/n/home12/rebecca.krall/.conda/envs/py3xg/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/n/home12/rebecca.krall/.conda/envs/py3xg/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
features = df[['vendor_id','passenger_count','pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude','pu_hour','wday','month','workday','precipitation','snowfall','snowdepth','total_distance','total_travel_time','jfk','lga']]
target = df['trip_duration']

In [4]:
tfeatures = tdf[['vendor_id','passenger_count','pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude','pu_hour','wday','month','workday','precipitation','snowfall','snowdepth','total_distance','total_travel_time','jfk','lga']]

In [5]:
def rmsle(evaluator,X,real):
    sum = 0.0
    predicted = evaluator.predict(X)
    print("Number predicted less than 0: {}".format(np.where(predicted < 0)[0].shape))

    predicted[predicted < 0] = 0
    for x in range(len(predicted)):
        p = np.log(predicted[x]+1)
        r = np.log(real[x]+1)
        sum = sum + (p-r)**2
    return (sum/len(predicted))**0.5

In [6]:
cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}
ind_params = {'learning_rate': 0.08, 'n_estimators': 100, 'seed':0, 'subsample': 0.75, 'colsample_bytree': 1}
optimized_GBM = GridSearchCV(xgboost.XGBRegressor(**ind_params), 
                            cv_params,scoring = rmsle, cv =4) 
optimized_GBM.fit(features, np.ravel(target))

Number predicted less than 0: (0,)
Number predicted less than 0: (0,)
Number predicted less than 0: (0,)
Number predicted less than 0: (0,)
Number predicted less than 0: (0,)
Number predicted less than 0: (0,)
Number predicted less than 0: (3,)
Number predicted less than 0: (0,)
Number predicted less than 0: (0,)
Number predicted less than 0: (0,)
Number predicted less than 0: (3,)
Number predicted less than 0: (0,)
Number predicted less than 0: (0,)
Number predicted less than 0: (0,)
Number predicted less than 0: (0,)
Number predicted less than 0: (0,)
Number predicted less than 0: (0,)
Number predicted less than 0: (1,)
Number predicted less than 0: (0,)
Number predicted less than 0: (0,)
Number predicted less than 0: (0,)
Number predicted less than 0: (0,)
Number predicted less than 0: (1,)
Number predicted less than 0: (0,)
Number predicted less than 0: (1,)
Number predicted less than 0: (0,)
Number predicted less than 0: (0,)
Number predicted less than 0: (0,)
Number predicted les

GridSearchCV(cv=4, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.08, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.75),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_depth': [3, 5, 7], 'min_child_weight': [1, 3, 5]},
       pre_dispatch='2*n_jobs', refit=True,
       scoring=<function rmsle at 0x2ab20d5ba0d0>, verbose=0)

In [7]:
print(optimized_GBM.grid_scores_)

[mean: 0.37750, std: 0.00359, params: {'max_depth': 3, 'min_child_weight': 1}, mean: 0.37759, std: 0.00351, params: {'max_depth': 3, 'min_child_weight': 3}, mean: 0.37800, std: 0.00308, params: {'max_depth': 3, 'min_child_weight': 5}, mean: 0.35776, std: 0.00301, params: {'max_depth': 5, 'min_child_weight': 1}, mean: 0.35801, std: 0.00307, params: {'max_depth': 5, 'min_child_weight': 3}, mean: 0.35778, std: 0.00334, params: {'max_depth': 5, 'min_child_weight': 5}, mean: 0.34564, std: 0.00297, params: {'max_depth': 7, 'min_child_weight': 1}, mean: 0.34685, std: 0.00366, params: {'max_depth': 7, 'min_child_weight': 3}, mean: 0.34769, std: 0.00342, params: {'max_depth': 7, 'min_child_weight': 5}]
